# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [42]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

from pyspark.sql import Window
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import countDistinct

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [3]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [4]:
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

# Question 1

Which page did user id "" (empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1
user_log.select(["page"]).where(user_log.userId == "").dropDuplicates().collect()

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [6]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(["auth", "registration"]).where(user_log.userId == "").dropDuplicates().collect()

[Row(auth='Logged Out', registration=None),
 Row(auth='Guest', registration=None)]

# Question 3

How many female users do we have in the data set?

In [7]:
# TODO: write your code to answer question 3
user_log.select("userId").filter(user_log.gender=='F').dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [15]:
# TODO: write your code to answer question 4
user_log.select("artist").groupby(user_log.artist).count().orderBy("count", ascending=0).collect()[:5]

[Row(artist=None, count=1653),
 Row(artist='Coldplay', count=83),
 Row(artist='Kings Of Leon', count=69),
 Row(artist='Florence + The Machine', count=52),
 Row(artist='BjÃ\x83Â¶rk', count=46)]

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [40]:
# TODO: write your code to answer question 5

# wrong answer
df_count = user_log.select(user_log["userID"]) \
    .filter((user_log.page=='Home') & (user_log.userId!="")) \
    .groupby(user_log["userId"]).count()
df_count.groupby().avg().show()

+-----------------+
|       avg(count)|
+-----------------+
|1.562396006655574|
+-----------------+



In [ ]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)